<h1>Colab Python Notebook For How Banks Can Save The World Pt 1</h1>

<h2>Applies Electra NLP model to company Bios to identify taxonomies</h2>

Downloads tensorflow based NLP model 'electra' fine tuned for Q&A

Creates input data using company bio as context, using a series of questions designed to pick up company Taxonomy (using a sample of EU TAG taxonomies) 

Runs predictions on the model using the company bio and questions. The model produces null odds in the form of Logits (https://towardsdatascience.com/logit-of-logistic-regression-understanding-the-fundamentals-f384152a33d1) - essentially for our purposes any score less than -3.0 is satisfactory to indicate that a company matches a given taxonomy. 



In [ ]:
%tensorflow_version 1.x

In [ ]:
import tensorflow.compat.v1 as tf
import tensorflow.compat.v1 as tensorflow
print(tensorflow.__version__)

1.15.2


In [ ]:
!git clone https://github.com/infospark/electra.git


Cloning into 'electra'...
remote: Enumerating objects: 378, done.
remote: Total 378 (delta 0), reused 0 (delta 0), pack-reused 378
Receiving objects: 100% (378/378), 11.88 MiB | 12.41 MiB/s, done.
Resolving deltas: 100% (249/249), done.


In [ ]:
cd /content/electra

/content/electra


# Set up Tensor Environment

In [ ]:
from google.colab import auth
auth.authenticate_user()

In [ ]:
import datetime
import json
import os
import pprint
import random
import string
import sys
import tensorflow as tf


assert 'COLAB_TPU_ADDR' in os.environ, 'ERROR: Not connected to a TPU runtime; please see the first cell in this notebook for instructions!'
TPU_ADDRESS = 'grpc://' + os.environ['COLAB_TPU_ADDR']
print('TPU address is => ', TPU_ADDRESS)


with tf.Session(TPU_ADDRESS) as session:
  print('TPU devices:')
  pprint.pprint(session.list_devices())

  # Upload credentials to TPU.
  with open('/content/adc.json', 'r') as f:
    auth_info = json.load(f)
  tf.contrib.cloud.configure_gcs(session, credentials=auth_info)
  # Now credentials are set for all future sessions on this TPU.

TPU address is =>  grpc://10.76.239.106:8470
TPU devices:
[_DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:CPU:0, CPU, -1, 8851134886854969880),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 17179869184, 16748168128226500254),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:0, TPU, 17179869184, 12850536239386584223),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:1, TPU, 17179869184, 11258504247144123985),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:2, TPU, 17179869184, 12404932437011996992),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:3, TPU, 17179869184, 770279234642390250),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:4, TPU, 17179869184, 7113252404208261920),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:5, TPU, 17179869184, 5131295798912748211),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:6, TPU, 17179869184, 15799142

In [ ]:
%%writefile dev.json
{
    "version": "v2.0",
    "data": [
        {
            "title": "Taxonomy",
            "paragraphs": [
                {
                    "qas": [
                        {
                            "question": "Is cement produced?",
                            "id": "q_taxonomy_cement_1",
                            "answers": [],
                            "is_impossible": false
                        },{
                            "question": "Is there cement business?",
                            "id": "q_taxonomy_cement_2",
                            "answers": [],
                            "is_impossible": false
                        },{
                            "question": "Is cement sold?",
                            "id": "q_taxonomy_cement_3",
                            "answers": [],
                            "is_impossible": false
                        },{
                            "question": "Is steel produced?",
                            "id": "q_taxonomy_steel_1",
                            "answers": [],
                            "is_impossible": false
                        },{
                            "question": "Is there steel business?",
                            "id": "q_taxonomy_steel_2",
                            "answers": [],
                            "is_impossible": false
                        },{
                            "question": "Is steel sold?",
                            "id": "q_taxonomy_steel_3",
                            "answers": [],
                            "is_impossible": false
                        },{
                            "question": "Is Aluminium produced?",
                            "id": "q_taxonomy_aluminium_1",
                            "answers": [],
                            "is_impossible": false
                        },{
                            "question": "Is there Aluminium business?",
                            "id": "q_taxonomy_aluminium_2",
                            "answers": [],
                            "is_impossible": false
                        },{
                            "question": "Is Aluminium sold?",
                            "id": "q_taxonomy_aluminium_3",
                            "answers": [],
                            "is_impossible": false
                        },{
                            "question": "Are fertilizers or nitrogen compounds produced?",
                            "id": "q_taxonomy_fertilizer_nitrogen_1",
                            "answers": [],
                            "is_impossible": false
                        },{
                            "question": "Is there Aluminium business?",
                            "id": "q_taxonomy_fertilizer_nitrogen_2",
                            "answers": [],
                            "is_impossible": false
                        },{
                            "question": "Is Aluminium sold?",
                            "id": "q_taxonomy_fertilizer_nitrogen_3",
                            "answers": [],
                            "is_impossible": false
                        },{
                            "question": "Are fertilizers or nitrogen compounds produced?",
                            "id": "q_taxonomy_fertilizer_nitrogen",
                            "answers": [],
                            "is_impossible": false
                        }
                    ],
                    "context": "HeidelbergCement is a German multinational building materials company headquartered in Heidelberg, Germany. It is a DAX corporation and is one of the largest building materials companies in the world. On 1 July 2016, HeidelbergCement AG completed the acquisition of a 45% shareholding in Italcementi.  That acquisition made HeidelbergCement the number one producer of construction aggregates, the number 2 in cement and number 3 in ready mixed concrete worldwide. In the 2020 Forbes Global 2000, HeidelbergCement was ranked as the 678th -largest public company in the world.[4] The enlarged group has activities in around 60 countries with 57,000 employees working at 3,000 production sites. HeidelbergCement operates 139 cement plants with an annual cement capacity of 176 million tonnes, more than 1,500 ready-mixed concrete production sites, and over 600 aggregates quarries.[5] HeidelbergCement is responsible for 0.08% of global industrial greenhouse gas emissions from 1751 to 2016[6] and thus one of the largest contributors to climate change, which bears substantial “risks to health, livelihoods, food security, water supply, human security, and economic growth”.[7]"

                }
            ]
        }
    ]
}

Writing dev.json


In [ ]:
!gsutil mv dev.json gs://jims_bert_test/electra_data_large/finetuning_data/squad/

Copying file://dev.json [Content-Type=application/json]...
Removing file://dev.json...

Operation completed over 1 objects/4.6 KiB.                                      


In [ ]:
!git pull
!gsutil rm gs://jims_bert_test/electra_data_large/models/electra_large/finetuning_tfrecords/squad_tfrecords/squad_dev.*
!python3 run_finetuning.py --data-dir gs://jims_bert_test/electra_data_large/ --model-name electra_large --hparams '{"tpu_name": "grpc://10.76.239.106:8470", "do_train": false, "do_eval": true, "model_size": "large", "task_names": ["squad"], "init_checkpoint": "gs://jims_bert_test/electra_data_large/models/electra_large/finetuning_models/squad_model_1", "use_tpu": "True", "num_tpu_cores":8}'
#!python3 run_finetuning.py --data-dir gs://jims_bert_test/electra_data/ --model-name electra_base --hparams '{"do_train": false, "do_eval": true, "model_size": "base", "task_names": ["squad"], "init_checkpoint": "gs://jims_bert_test/electra_data/models/electra_base/finetuning_models/squad_model_1", "use_tpu": "False"}'

Already up to date.
Removing gs://jims_bert_test/electra_data_large/models/electra_large/finetuning_tfrecords/squad_tfrecords/squad_dev.metadata...
Removing gs://jims_bert_test/electra_data_large/models/electra_large/finetuning_tfrecords/squad_tfrecords/squad_dev.tfrecord...
/ [2 objects]                                                                   
Operation completed over 2 objects.                                              
Config: model=electra_large, trial 1/1
answerable_classifier True
answerable_uses_start_logits True
answerable_weight 0.5
beam_size 20
data_dir gs://jims_bert_test/electra_data_large/
debug False
do_eval True
do_lower_case True
do_train False
doc_stride 128
double_unordered True
embedding_size None
eval_batch_size 32
exact_file <built-in method format of str object at 0x7fada7c138a0>
f1_file <built-in method format of str object at 0x7fada7c13930>
gcp_project None
init_checkpoint gs://jims_bert_test/electra_data_large/models/electra_large/finetuning_model

In [ ]:
!gsutil cat gs://jims_bert_test/electra_data_large/models/electra_large/results/squad_qa/squad_null_odds.json

{"q_taxonomy_cement_1": -6.846512317657471, "q_taxonomy_cement_2": -7.090182304382324, "q_taxonomy_cement_3": -3.9172158241271973, "q_taxonomy_steel_1": 5.059553146362305, "q_taxonomy_steel_2": 0.8183633685112, "q_taxonomy_steel_3": 6.963508605957031, "q_taxonomy_aluminium_1": 6.155025482177734, "q_taxonomy_aluminium_2": 1.4605306386947632, "q_taxonomy_aluminium_3": 7.4915242195129395, "q_taxonomy_fertilizer_nitrogen_1": 6.001425266265869, "q_taxonomy_fertilizer_nitrogen_2": 1.4605306386947632, "q_taxonomy_fertilizer_nitrogen_3": 7.4915242195129395, "q_taxonomy_fertilizer_nitrogen": 6.001425266265869}